In [1]:
import os
import sys

# Current working directory of the notebook
current_dir = os.getcwd()

# Path to the folder one level above
external_path = os.path.abspath(os.path.join(current_dir, '..'))

if external_path not in sys.path:
    sys.path.insert(0, external_path)

In [2]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

from qsarcons.consensus import RandomSearchRegressor, SystematicSearchRegressor, GeneticSearchRegressor
from qsarcons.consensus import RandomSearchClassifier, SystematicSearchClassifier, GeneticSearchClassifier

### Load built QSAR models

The input data for consensus/stacking methods is a pandas DataFrame where each column is a model prediction. For benchmarking proposes, we need two tables:

- Validation set predictions – they are used to find an optimal consensus or stacking ensemble
- Test set predictions - is needed to evaluate the performance of the consensus/stacking

### Input file format

The input CSV files (`train.csv` and `test.csv`) are expected to have the following format:
- **Column 0** – SMILES (or any other molecule ID)
- **Column 1** – Observed/Experimental property/activity values
- **Columns 2+** – Model predictions (from individual QSAR models)

In [3]:
df_val = pd.read_csv("CHEMBL1785/val.csv", )
df_test = pd.read_csv("CHEMBL1785/test.csv")

In [4]:
df_test.head(5)

,SMILES,Y_TRUE,atompair-count|BayesianRidge,atompair-count|DecisionTreeRegressor,atompair-count|ElasticNet,atompair-count|HuberRegressor,atompair-count|KNeighborsRegressor,atompair-count|Lasso,atompair-count|MLPRegressor,atompair-count|PLSRegression,...,topological|ElasticNet,topological|HuberRegressor,topological|KNeighborsRegressor,topological|Lasso,topological|MLPRegressor,topological|PLSRegression,topological|RandomForestRegressor,topological|RidgeRegression,topological|SVR,topological|XGBRegressor
0,COCCOc1nc(NS(=O)(=O)NCc2ccccc2)c(Oc2ccccc2OC)c...,6.256,6.610020,6.306200,6.890746,7.309935,6.475773,6.829488,6.304784,6.444161,...,6.812998,5.728646,6.475603,6.468520,5.356423,6.243217,6.494189,6.314992,6.418615,6.628993
1,COc1cnc(O[C@H](C(=O)O)[C@@]2(c3ccccc3)NCC(=O)N...,5.034,5.543897,5.364375,5.313822,5.699822,5.631566,5.259852,4.507620,5.451443,...,5.720473,5.176263,5.306299,5.679028,5.425185,5.808187,5.386879,5.766536,5.120113,5.258121
2,CCOc1ccc2c(c1)c(=O)c(Cc1cccc(C(=O)O)c1)c(C(=O)...,5.506,6.069682,5.642833,6.222900,6.447204,5.950810,6.151965,6.365187,5.857042,...,5.792353,6.120300,5.804565,5.804876,6.102401,5.799607,5.865963,5.821107,6.008066,5.883258
3,CCOc1ccc2c(c1)c(-c1ccc(OC)cc1OC)c(C(=O)O)c(=O)...,6.268,6.046365,7.001400,6.301070,6.842113,6.001984,6.292202,6.563304,5.848380,...,5.795346,6.059051,5.908972,5.922968,6.206772,5.821332,5.954252,5.898207,5.992958,6.383691
4,Cc1ccc(S(=O)(=O)Nc2onc(C)c2Br)cc1,5.398,4.952641,5.333167,5.100456,5.492576,4.598690,5.098454,5.083330,4.882305,...,4.951271,5.060801,5.131640,4.970824,5.072250,4.818004,5.091551,4.989698,5.410053,5.154193


In [5]:
# skip first two columns (smiles and true property value)
x_val, true_val = df_val.iloc[:, 2:], df_val.iloc[:, 1]
x_test, true_test = df_test.iloc[:, 2:], df_test.iloc[:, 1]

### Build Consensus / Stacking

For consensus methods, there are two main parameters that must be predefined:

**1. Metric**

This is the *prediction accuracy metric* to be optimized. The following regression metrics are currently supported:

- ``r2`` – Coefficient of determination
- ``rmse`` – Root mean squared error
- ``mae`` – Mean absolute error
- ``spearmanr`` – Spearman rank correlation coefficient

For benchmarking purposes, it is recommended to set the optimized metric to match the one used for estimating prediction accuracy on the test set.

**2. Consensus Size**

The *consensus size* determines how many models are included in the consensus. It can be defined in two ways:

- **Manual selection**: for example, ``cons_size=10``
- **Automatic selection**: e.g., ``cons_size="auto", cons_size_candidates=[3, 5, 7, 9, 12, 15]``, where the best consensus size is chosen based on metric values computed on the validation set.

In [6]:
def calc_accuracy(y_true, y_pred):
    return r2_score(y_true, y_pred)

In [7]:
metric = "auto"
cons_size = "auto"

df_comp = pd.DataFrame()

### Best model

Single best model

In [8]:
cons_searcher = SystematicSearchRegressor(cons_size=1, metric=metric)
best_cons = cons_searcher.run(x_val, true_val)
best_cons.to_list()

['atompair-count|XGBRegressor']

In [9]:
# make val and test predictions
pred_val = cons_searcher._consensus_predict(x_val[best_cons])
pred_test = cons_searcher._consensus_predict(x_test[best_cons])

# write prediction accuracy metric
df_comp.loc["val", "Best"] = calc_accuracy(true_val, pred_val)
df_comp.loc["test", "Best"] = calc_accuracy(true_test, pred_test)

# display current results
df_comp

,Best
val,0.806120
test,0.823806


### Random consensus

In a random search for optimal consensus the subset of models is chosen randomly and its prediction accuracy is computed. This procedure is repeated ``n_iter`` times, then the subset (consensus) with the highest prediction accuracy on the validation set is chosen as the best consensus.

In [10]:
cons_searcher = RandomSearchRegressor(cons_size=cons_size, n_iter=1000, metric=metric)
best_cons = cons_searcher.run(x_val, true_val)
best_cons.to_list()

['pharm2D-gobbi|ElasticNet',
 'atompair-count|Lasso',
 'ecfp|XGBRegressor',
 'desc2D|SVR',
 'atompair-count|RandomForestRegressor']

In [11]:
# make val and test predictions
pred_val = cons_searcher._consensus_predict(x_val[best_cons])
pred_test = cons_searcher._consensus_predict(x_test[best_cons])

# write prediction accuracy metric
df_comp.loc["val", "Random"] = calc_accuracy(true_val, pred_val)
df_comp.loc["test", "Random"] = calc_accuracy(true_test, pred_test)

# display current results
df_comp

,Best,Random
val,0.806120,0.807083
test,0.823806,0.837412


### Systematic consensus

In a systematic search for optimal consensus, all models are sorted according to their prediction accuracy on the validation set, and then the first *N* models (*N*=``cons_size``) are chosen as the best consensus.

In [12]:
cons_searcher = SystematicSearchRegressor(cons_size=cons_size, metric=metric)
best_cons = cons_searcher.run(x_val, true_val)
best_cons.to_list()

['atompair-count|XGBRegressor',
 'atompair-count|RandomForestRegressor',
 'ecfp-count|XGBRegressor',
 'fcfp-count|XGBRegressor',
 'ecfp-count|RandomForestRegressor',
 'topological|RandomForestRegressor',
 'atompair-count|BayesianRidge',
 'ecfp|XGBRegressor',
 'fcfp|XGBRegressor',
 'topological|XGBRegressor',
 'fcfp-count|RandomForestRegressor',
 'rdkit|SVR',
 'pharm2D-cats|XGBRegressor',
 'atompair-count|RidgeRegression',
 'rdkit|BayesianRidge']

In [13]:
# make val and test predictions
pred_val = cons_searcher._consensus_predict(x_val[best_cons])
pred_test = cons_searcher._consensus_predict(x_test[best_cons])

# write prediction accuracy metric
df_comp.loc["val", "Systematic"] = calc_accuracy(true_val, pred_val)
df_comp.loc["test", "Systematic"] = calc_accuracy(true_test, pred_test)

# display current results
df_comp

,Best,Random,Systematic
val,0.806120,0.807083,0.809657
test,0.823806,0.837412,0.846218


### Genetic consensus

In a genetic search for optimal consensus, the best consensus of models is being founded using a genetic algorithm. In this case, the objective function is a prediction accuracy for the consensus (individual). 

In [14]:
cons_searcher = GeneticSearchRegressor(cons_size=cons_size, n_iter=50, pop_size=50, mut_prob=0.2, metric=metric)
best_cons = cons_searcher.run(x_val, true_val)
best_cons.to_list()

['atompair-count|XGBRegressor',
 'cats2d|RandomForestRegressor',
 'rdkit|MLPRegressor',
 'atompair-count|RidgeRegression',
 'pharm2D-gobbi|PLSRegression',
 'ecfp-count|XGBRegressor',
 'topological|XGBRegressor']

In [15]:
# make val and test predictions
pred_val = cons_searcher._consensus_predict(x_val[best_cons])
pred_test = cons_searcher._consensus_predict(x_test[best_cons])

# write prediction accuracy metric
df_comp.loc["val", "Genetic"] = calc_accuracy(true_val, pred_val)
df_comp.loc["test", "Genetic"] = calc_accuracy(true_test, pred_test)

# display current results
df_comp

,Best,Random,Systematic,Genetic
val,0.806120,0.807083,0.809657,0.819432
test,0.823806,0.837412,0.846218,0.847721


### Classifier consensus

Classifier consensus

In [16]:
from sklearn.metrics import accuracy_score, roc_auc_score

In [17]:
def calc_accuracy(y_true, y_pred):
    return roc_auc_score(y_true, y_pred)

In [18]:
metric = "auto"
cons_size = "auto"

df_comp = pd.DataFrame()

In [19]:
cons_class_methods = [
    ("Best", SystematicSearchClassifier(cons_size=1, metric=metric)),         
    ("Random", RandomSearchClassifier(cons_size=cons_size, n_iter=1000, metric=metric)),       
    ("Systematic", SystematicSearchClassifier(cons_size=cons_size, metric=metric)),
    ("Genetic", GeneticSearchClassifier(cons_size=cons_size, n_iter=50, pop_size=50, mut_prob=0.2, metric=metric))
]

In [20]:
# load model predictions
df_val = pd.read_csv("../bench_hopt/tdc/herg_clf/val.csv")
df_test = pd.read_csv("../bench_hopt/tdc/herg_clf/test.csv")

# skip first two columns (smiles and true property value)
x_val, true_val = df_val.iloc[:, 2:], df_val.iloc[:, 1]
x_test, true_test = df_test.iloc[:, 2:], df_test.iloc[:, 1]

In [21]:
for name, cons_searcher in cons_class_methods:
    # run search
    best_cons = cons_searcher.run(x_val, true_val)
    
    # make val and test predictions
    pred_val = cons_searcher._consensus_predict(x_val[best_cons])
    pred_test = cons_searcher._consensus_predict(x_test[best_cons])
    
    # # write prediction accuracy metric
    df_comp.loc["val", name] = roc_auc_score(true_val, pred_val)
    df_comp.loc["test", name] = roc_auc_score(true_test, pred_test)

In [22]:
df_comp.round(3)

,Best,Random,Systematic,Genetic
val,0.813,0.842,0.835,0.873
test,0.641,0.695,0.633,0.704
